# <center> US pollution analysis

### Introduction

The US Pollution Data project leverages a comprehensive dataset covering air pollution measurements across the United States — consisting of over 1.4 million observations and around 28 variables that record concentrations of major pollutants such as nitrogen dioxide (NO₂), sulfur dioxide (SO₂), carbon monoxide (CO), and ozone (O₃). 


This dataset spans several years and states, allowing detailed spatio-temporal analysis of pollutant levels, seasonal trends, and geographic variation. The primary objective of this project is to transform this raw data into clean, analysis-ready formats (e.g., Parquet), conduct exploratory data analysis (EDA) to uncover patterns and insights, and — where possible — apply machine learning techniques to forecast pollutant concentrations, classify air‑quality levels, or identify key factors driving pollution.

Given the public health importance of air quality, this project has the potential not only to improve our understanding of pollution trends in the U.S., but also to inform policy, raise awareness, or support predictive systems that warn populations about deteriorating air conditions.

The goal of this project is to:

* Clean and transform the raw data (e.g., into Parquet)
* Perform EDA to uncover trends, seasonal patterns, and geographic variation
* Apply ML techniques to forecast pollutant levels, classify air quality, and analyze feature importance

By improving our understanding of air quality trends, this project supports public health insights and data-driven policy decisions.

Dataset content:
* State Code: Numeric code representing the U.S. state
* County Code:	Numeric code for the county within the state
* Site Num:	Identifier for the air monitoring site
* Address:	Street address of the monitoring station
* State:	Full name of the U.S. state
* County:	Name of the county
* City:	City where the measurement site is located
* Date Local:	Date of the observation (YYYY-MM-DD)
* NO2 Units:	Units used for nitrogen dioxide measurements
* NO2 Mean:	Daily average NO₂ concentration
* NO2 1st Max Value:	Highest NO₂ value recorded that day
* NO2 1st Max Hour:	Hour when the highest NO₂ was recorded
* NO2 AQI:	Air Quality Index for NO₂ on that day
* O3 Units:	Units used for ozone measurements
* O3 Mean:	Daily average ozone concentration
* O3 1st Max Value:	Highest O₃ value recorded that day
* O3 1st Max Hour:	Hour when the highest O₃ was recorded
* O3 AQI:	Air Quality Index for O₃ on that day
* SO2 Units:	Units used for sulfur dioxide measurements
* SO2 Mean:	Daily average SO₂ concentration
* SO2 1st Max Value:	Highest SO₂ value recorded that day
* SO2 1st Max Hour:	Hour when the highest SO₂ was recorded
* SO2 AQI:	Air Quality Index for SO₂ on that day
* CO Units:	Units used for carbon monoxide measurements
* CO Mean:	Daily average CO concentration
* CO 1st Max Value:	Highest CO value recorded that day
* CO 1st Max Hour:	Hour when the highest CO was recorded
* CO AQI:	Air Quality Index for CO on that day

---

### 1. EDA and Initial data visualisation

First, all necessary libraries are imported

In [1]:
import os                          #import os for operating system interactions
import pandas as pd                 #import Pandas for data manipulation
import numpy as np                  #import Numpy for numerical operations
import matplotlib.pyplot as plt     #import Matplotlib for data visualization
import seaborn as sns               #import Seaborn for statistical data visualization
from plotly.subplots import make_subplots  #import Plotly subplots for creating complex figures
import plotly.express as px         #import Plotly Express for interactive visualizations
import plotly.graph_objects as go   #import Plotly Graph Objects for detailed figure customization

In [2]:
sns.set(style="whitegrid")                  # Set Seaborn style for plots
plt.rcParams["figure.figsize"] = (10,6)     # Set default figure size for Matplotlib plots

#### 1.1. ETL and EDA


In this section EDA, including data load and cleaning, is performed. As a first step, data set is loaded into DataFrame

In [3]:
df = pd.read_parquet("../data/raw/pollution_dataset.parquet", engine="pyarrow")
df.head()

,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,1.145833,4.2,21,NaN
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,0.878947,2.2,23,25.0
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,1.145833,4.2,21,NaN
3,3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,0.878947,2.2,23,25.0
4,4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,Parts per billion,1.958333,3.0,22,4.0,Parts per million,0.850000,1.6,23,NaN


In [4]:
df. drop(columns=['Unnamed: 0'], inplace=True)  # Drop unnecessary column
df.head()

,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,NO2 Mean,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,19.041667,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,1.145833,4.2,21,NaN
1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,19.041667,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,0.878947,2.2,23,25.0
2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,19.041667,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,1.145833,4.2,21,NaN
3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,19.041667,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,0.878947,2.2,23,25.0
4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,22.958333,...,Parts per billion,1.958333,3.0,22,4.0,Parts per million,0.850000,1.6,23,NaN


In the following subsection initial data set inspection is performed. Here the shape and Info of DataFrame are shown

In [5]:
print(df.shape)                     # Print the shape of the DataFrame           
print(df.info())                    # Print concise summary of the DataFrame            
print(df.dtypes)                    # Print data types of each column

(1746661, 28)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1746661 entries, 0 to 1746660
Data columns (total 28 columns):
 #   Column             Dtype  
---  ------             -----  
 0   State Code         int64  
 1   County Code        int64  
 2   Site Num           int64  
 3   Address            object 
 4   State              object 
 5   County             object 
 6   City               object 
 7   Date Local         object 
 8   NO2 Units          object 
 9   NO2 Mean           float64
 10  NO2 1st Max Value  float64
 11  NO2 1st Max Hour   int64  
 12  NO2 AQI            int64  
 13  O3 Units           object 
 14  O3 Mean            float64
 15  O3 1st Max Value   float64
 16  O3 1st Max Hour    int64  
 17  O3 AQI             int64  
 18  SO2 Units          object 
 19  SO2 Mean           float64
 20  SO2 1st Max Value  float64
 21  SO2 1st Max Hour   int64  
 22  SO2 AQI            float64
 23  CO Units           object 
 24  CO Mean            float64
 25  CO 1

In [6]:
df.dtypes.value_counts()         # Count occurrences of each data type

float64    10
int64       9
object      9
Name: count, dtype: int64

As it shown above dataset consists of 1746661 entries and 28 columns. Also dataset contains 10 float columns, 9 integer and 9 categorical columns. 

In the next steps DataFrame is checked for any incosistencies(dublicates, missing value and etc.)

In [7]:
df.isnull().sum()           # Check for missing values in each column

State Code                0
County Code               0
Site Num                  0
Address                   0
State                     0
County                    0
City                      0
Date Local                0
NO2 Units                 0
NO2 Mean                  0
NO2 1st Max Value         0
NO2 1st Max Hour          0
NO2 AQI                   0
O3 Units                  0
O3 Mean                   0
O3 1st Max Value          0
O3 1st Max Hour           0
O3 AQI                    0
SO2 Units                 0
SO2 Mean                  0
SO2 1st Max Value         0
SO2 1st Max Hour          0
SO2 AQI              872907
CO Units                  0
CO Mean                   0
CO 1st Max Value          0
CO 1st Max Hour           0
CO AQI               873323
dtype: int64

The dataset misses 872907 and 873323 values in SO2 AQI and CO AQI columns respectively. This is a big amount of missing data to just remove lines. Instead, these missing values can be calculated, as far as other columns have no missing points.

Air Quality Index (AQI) is calculated by converting measured pollutant concentrations (e.g., SO₂, CO, NO₂, O₃, PM₂.₅, PM₁₀) into a standardized scale (usually 0–500) using breakpoints.
Core AQI Formula

Each pollutant gets its own AQI number. The final AQI for the city/location is the highest of all pollutants.

Each government sets concentration ranges for each pollutant.
Example (US EPA standard):
SO₂ 1-hour breakpoints (ppb)
AQI Range	                SO₂ (ppb)
0–50 (Good)	                0–35
51–100 (Moderate)	        36–75
101–150 (Unhealthy SG)	    76–185
151–200 (Unhealthy)	        186–304
201–300 (Very Unhealthy)    305–604

CO 8-hour breakpoints (ppm)
AQI Range	CO (ppm)
0–50	    0.0–4.4
51–100	    4.5–9.4
101–150	    9.5–12.4
151–200	    12.5–15.4
201–300	    15.5–30.4

Below SO2 AQI column calculated based on given values in dedicated SO2 columns. 

In [8]:

def calculate_so2_aqi(C):
    """
    Calculate SO2 AQI using expanded breakpoint intervals (Option C).
    This avoids NA values from strict EPA bins.
    
    C : float 
        1-hour SO2 concentration in ppb
    """

    if pd.isna(C):
        return np.nan

    # ===== Expanded breakpoints ensuring continuous coverage =====
    if 0 <= C <= 35.999:
        Clow, Chigh = 0, 35
        Ilow, Ihigh = 0, 50

    elif 36 <= C <= 75.999:
        Clow, Chigh = 36, 75
        Ilow, Ihigh = 51, 100

    elif 76 <= C <= 185.999:
        Clow, Chigh = 76, 185
        Ilow, Ihigh = 101, 150

    elif 186 <= C <= 304.999:
        Clow, Chigh = 186, 304
        Ilow, Ihigh = 151, 200

    elif 305 <= C <= 604.999:
        Clow, Chigh = 305, 604
        Ilow, Ihigh = 201, 300

    else:
        # Out of range but we extend for safety
        return np.nan

    # ===== AQI Formula =====
    aqi = ((Ihigh - Ilow) / (Chigh - Clow)) * (C - Clow) + Ilow
    return round(aqi, 1)


In [9]:
# Function to add SO2 AQI column to DataFrame
def add_so2_aqi_column(df, col_name="SO2 1st Max Value"):
    """
    df : pandas dataframe  
    col_name : column containing SO2 1-hour max in ppb
    """
    df = df.copy()
    df["SO2 AQI"] = df[col_name].apply(calculate_so2_aqi)
    return df

In [10]:
df = add_so2_aqi_column(df, "SO2 1st Max Value")
df["SO2 AQI"].isna().sum()

np.int64(8286)

And check column  and values

In [11]:
print(df[["SO2 1st Max Value", "SO2 AQI"]].head())

   SO2 1st Max Value  SO2 AQI
0                9.0     12.9
1                9.0     12.9
2                6.6      9.4
3                6.6      9.4
4                3.0      4.3


And CO AQI calculation

In [12]:
import numpy as np
import pandas as pd

def calculate_co_aqi(C):
    """
    Calculate CO AQI using expanded breakpoint intervals (Option C).
    This ensures continuous coverage with NO missing AQI values.

    C : float 
        8-hour CO concentration in ppm
    """

    if pd.isna(C):
        return np.nan

    # ===== Expanded breakpoints =====
    if 0.0 <= C <= 4.499:
        Clow, Chigh = 0.0, 4.4
        Ilow, Ihigh = 0, 50

    elif 4.5 <= C <= 9.499:
        Clow, Chigh = 4.5, 9.4
        Ilow, Ihigh = 51, 100

    elif 9.5 <= C <= 12.499:
        Clow, Chigh = 9.5, 12.4
        Ilow, Ihigh = 101, 150

    elif 12.5 <= C <= 15.499:
        Clow, Chigh = 12.5, 15.4
        Ilow, Ihigh = 151, 200

    elif 15.5 <= C <= 30.499:
        Clow, Chigh = 15.5, 30.4
        Ilow, Ihigh = 201, 300

    else:
        return np.nan  # extremely high or wrong units

    # ===== AQI Formula =====
    aqi = ((Ihigh - Ilow) / (Chigh - Clow)) * (C - Clow) + Ilow
    return round(aqi, 1)


In [13]:
def add_co_aqi_column(df, col="CO Mean"):
    df = df.copy()
    df["CO AQI"] = df[col].apply(calculate_co_aqi)
    return df

In [14]:
df = add_co_aqi_column(df, "CO Mean")
df["CO AQI"].isna().sum()

np.int64(1064)

Verifying imputations

In [15]:
df.isnull().sum()           # Check for missing values in each column

State Code              0
County Code             0
Site Num                0
Address                 0
State                   0
County                  0
City                    0
Date Local              0
NO2 Units               0
NO2 Mean                0
NO2 1st Max Value       0
NO2 1st Max Hour        0
NO2 AQI                 0
O3 Units                0
O3 Mean                 0
O3 1st Max Value        0
O3 1st Max Hour         0
O3 AQI                  0
SO2 Units               0
SO2 Mean                0
SO2 1st Max Value       0
SO2 1st Max Hour        0
SO2 AQI              8286
CO Units                0
CO Mean                 0
CO 1st Max Value        0
CO 1st Max Hour         0
CO AQI               1064
dtype: int64

As it can be seen ther are still missing values. Verifying values.

In [16]:
missing_so2 = df[df["SO2 AQI"].isna()][["SO2 Units", "SO2 Mean", "SO2 1st Max Value"]]
missing_co  = df[df["CO AQI"].isna()][["CO Units", "CO Mean", "CO 1st Max Value"]]

print("Missing SO2 samples:")
print(missing_so2.head())

print("Missing CO samples:")
print(missing_co.head())

Missing SO2 samples:
                 SO2 Units  SO2 Mean  SO2 1st Max Value
1004801  Parts per billion -0.428571               -0.3
1004802  Parts per billion -0.428571               -0.3
1091334  Parts per billion -0.391667               -0.3
1091335  Parts per billion -0.391667               -0.3
1091336  Parts per billion -0.375000               -0.3
Missing CO samples:
                  CO Units   CO Mean  CO 1st Max Value
1116634  Parts per million -0.015385               0.0
1116636  Parts per million -0.015385               0.0
1116638  Parts per million -0.019048               0.1
1116639  Parts per million -0.009524               0.1
1116640  Parts per million -0.019048               0.1


SO2 1stMax value = 35.3 ppb, which is in range 0–35, but 35.3 is slightly above 35, outside the defined interval.

EPA ranges MUST be exact, and 35.3 is not considered valid for AQI. So SO₂ values between 35.1 and 35.9 produce NaN by design. 
EPA rule:
Breakpoints are inclusive, and measurement rounding rules apply, but the datasets often contain decimals that cross boundaries.

“CO Mean” ~ 4.47 ppm → falls in correct interval. Values like 4.47 ppm fall exactly on the boundary between AQI bins

EPA boundaries are strict:
* 0.0–4.4 → AQI 0–50
* 4.5–9.4 → AQI 51–100
* 4.47 is neither ≤4.4 nor ≥4.5 → does not match any interval → NaN

And recalculating values for AQIs. Expand intervals (industry workaround) and turn EPA breakpoint edges into continuous. This completely eliminates NaN and remains realistic.